# Required imports

In [1]:
import os
import json
import sys
import base64
import requests
from IPython.display import Image, display

# Install the necessary packages

In [2]:
!{sys.executable} -m pip install oscar-python
!pip install liboidcagent
from oscar_python.client import Client

# Function to decode base64 content

In [3]:
def decode_base64(data):
    return base64.b64decode(data).decode('utf-8')

# Read environment variables

In [4]:
token_file_path = os.getenv("TOKEN_FILE_PATH")
image_file_path = os.getenv("IMAGE_FILE_PATH")
output_dir = os.getenv("OUTPUT_DIR", "./output/images")
service_name = os.getenv("SERVICE_NAME", "grayify")

# Read and decode the access token

In [5]:
with open(token_file_path, 'r') as token_file:
    encoded_token = token_file.read()
access_token = decode_base64(encoded_token)

# OSCAR client configuration

In [6]:
options_oidc = {
    'cluster_id': 'oscar-egi-cloud',
    'endpoint': 'https://inference.cloud.ai4eosc.eu',
    'oidc_token': access_token,
    'ssl': 'True'
}
oscar_client = Client(options=options_oidc)

# Check if the service exists

In [7]:
try:
    service = oscar_client.get_service(service_name)
    print(f"Service '{service_name}' already exists.")
except Exception as e:
    print(f"Service '{service_name}' does not exist, creating...")
    try:
        oscar_client.create_service(f"{service_name}.yaml")
        print("Service created successfully.")
    except Exception as ex:
        print(f"Error creating service: {ex}")

Service 'plant-classification' already exists.


# Run the service

In [8]:
response = oscar_client.run_service(service_name, input=image_file_path, output=output_dir)

# Process the response

In [9]:
if response.status_code == 200:
    # Process grayify service response
    data = response.text
    # No need to create a new directory, as output_file already includes the correct path
    # os.makedirs('./output/b64', exist_ok=True)  # Esta línea ya no es necesaria
    # Use the output_file variable directly
    print(data)
    with open(output_dir, 'w') as f:
        f.write(data)
else:
    print(f"Error in the execution of 'grayify' service: {response.status_code} {response.text}")

eydsYWJlbHMnOiBbJ1RhcmF4YWN1bSBlcnl0aHJvc3Blcm11bScsICdBZ29zZXJpcyBncmFuZGlmbG9yYScsICdUYXJheGFjdW0gb2ZmaWNpbmFsZScsICdFcmlvcGhvcnVtIHZhZ2luYXR1bScsICdQaWxvc2VsbGEgb2ZmaWNpbmFydW0nXSwgJ3Byb2JhYmlsaXRpZXMnOiBbMC4zMzA0MDY0MjczODM0MjI4NSwgMC4xOTU1NjAzNjU5MTUyOTg0NiwgMC4xNTIzOTI0MDIyOTEyOTc5LCAwLjA2MTg0Njk3MTUxMTg0MDgyLCAwLjA0Nzc4NjgyNDQwNTE5MzMzXSwgJ2xhYmVsc19pbmZvJzogWycnLCAnJywgJycsICcnLCAnJ10sICdsaW5rcyc6IHsnR29vZ2xlIEltYWdlcyc6IFsnaHR0cHM6Ly93d3cuZ29vZ2xlLmVzL3NlYXJjaD90Ym09aXNjaCZxPVRhcmF4YWN1bStlcnl0aHJvc3Blcm11bScsICdodHRwczovL3d3dy5nb29nbGUuZXMvc2VhcmNoP3RibT1pc2NoJnE9QWdvc2VyaXMrZ3JhbmRpZmxvcmEnLCAnaHR0cHM6Ly93d3cuZ29vZ2xlLmVzL3NlYXJjaD90Ym09aXNjaCZxPVRhcmF4YWN1bStvZmZpY2luYWxlJywgJ2h0dHBzOi8vd3d3Lmdvb2dsZS5lcy9zZWFyY2g/dGJtPWlzY2gmcT1FcmlvcGhvcnVtK3ZhZ2luYXR1bScsICdodHRwczovL3d3dy5nb29nbGUuZXMvc2VhcmNoP3RibT1pc2NoJnE9UGlsb3NlbGxhK29mZmljaW5hcnVtJ10sICdXaWtpcGVkaWEnOiBbJ2h0dHBzOi8vZW4ud2lraXBlZGlhLm9yZy93aWtpL1RhcmF4YWN1bV9lcnl0aHJvc3Blcm11bScsICdodHRwczovL2VuLndpa2lwZWRpYS5vcmcv